In [ ]:
from twarc import Twarc
import pandas as pd
import os
import json

consumer_key = 'rS6Uk2wrCB5v5TlwvtwM5bo5B'
consumer_secret = 'qyPDlCUROwwc2DMvbQARzdv6TQ4wyzMFQ3h5bKXRp6qTc8i3aM'
access_token = '267086736-4mMLPnPDux7r6djSuPHmd1NvlV4vZtj5Aq7DrIeH'
access_token_secret = '9kqN1ADnT6MB2Y9f24EdiDBxQ3oMRS7zQKZV0sGpEuXIZ'

t_inst = Twarc(consumer_key,consumer_secret,access_token, access_token_secret)

num_tweets=0
test = 0
# change this to different month such as 'tweets_ids/.../2020-10'
folder = '2020-08/'
for filename in os.listdir(folder):
    f = os.path.join(folder, filename)
    hour = filename[-6:-4]
    day = filename[-9:-7]
    month = filename[-12:-10]
    print(f)
    data = []
    for tweet in t_inst.hydrate(open(f)):
        num_tweets += 1
        # skips tweet if it doesn't have any location info 
        if tweet['place'] is None and tweet['user']['location'] == '':
            continue
        if tweet['place'] is None:
            place = None
            country = None
        else:
            place = tweet['place']['full_name']
            country = tweet['place']['country']

        data.append([tweet['id'], tweet['full_text'], tweet['user']['location'],place,country,tweet['retweeted'],tweet['retweet_count'],tweet['favorited'],tweet['favorite_count'],tweet['lang'],f'{month}-{day}-{hour}',tweet['user']['verified']])
        if num_tweets % 5000 == 0:
            print('Number of tweets hydrated:{}'.format(num_tweets))
           
    
    tweetdf = pd.DataFrame(data,columns=['id', 'text', 'location','place','country','retweeted','retweet_count','favorited','favorite_count','language','date','verified'])
    tweetdf.to_csv(f'df{month}-{day}-{hour}.csv',sep=';',index=False,header=True,encoding='utf-8')

2020-08/us-presidential-tweet-id-2020-08-01-00.txt
Number of tweets hydrated:5000
Number of tweets hydrated:15000
Number of tweets hydrated:20000


Number of tweets hydrated:25000
Number of tweets hydrated:30000
Number of tweets hydrated:40000
Number of tweets hydrated:45000


Number of tweets hydrated:60000
Number of tweets hydrated:70000


Number of tweets hydrated:75000
Number of tweets hydrated:80000
2020-08/us-presidential-tweet-id-2020-08-01-01.txt
Number of tweets hydrated:85000
Number of tweets hydrated:90000
Number of tweets hydrated:95000


Number of tweets hydrated:115000
Number of tweets hydrated:120000


Number of tweets hydrated:125000
Number of tweets hydrated:130000
Number of tweets hydrated:135000


Number of tweets hydrated:150000
Number of tweets hydrated:160000
2020-08/us-presidential-tweet-id-2020-08-01-02.txt
Number of tweets hydrated:175000


Number of tweets hydrated:180000
Number of tweets hydrated:190000
Number of tweets hydrated:195000


Number of tweets hydrated:205000
Number of tweets hydrated:215000
Number of tweets hydrated:220000


2020-08/us-presidential-tweet-id-2020-08-01-03.txt
Number of tweets hydrated:235000


In [2]:
import pandas as pd
import nltk
import os
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
sid = SentimentIntensityAnalyzer()
pd.options.display.max_colwidth = 10000


[nltk_data] Downloading package vader_lexicon to C:\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [24]:
df = pd.read_csv('df11-02-12.csv', sep=';').dropna(subset=['location'])
df['text'] = df['text'].str.lower()

In [25]:
states = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New York', 'New Mexico', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']
stateCodes = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']
stateMapping = {'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California', 'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia', 'HI': 'Hawaii', 'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland', 'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi', 'MO': 'Missouri', 'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NY': 'New York', 'NM': 'New Mexico', 'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio', 'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina', 'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT':  'Utah', 'VT': 'Vermont', 'VA': 'Virginia', 'WA': 'Washington', 'WV':  'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming'}

In [26]:
def sentimentVerdict(sentiment):
    if sentiment['compound'] >= 0.05:
        return "Positive"
    elif sentiment['compound'] <= -0.05:
        return "Negative"
    else:
        return "Neutral"

In [27]:
df['sentiment_scores'] = df['text'].apply(lambda x: sid.polarity_scores(x))
df['sentiment'] = df['sentiment_scores'].apply(lambda x: sentimentVerdict(x))
print(df['sentiment'].value_counts())

Positive    14359
Neutral     13774
Negative    13381
Name: sentiment, dtype: int64


In [28]:
df_terms = pd.read_csv('search_terms.csv', sep=';')
df_terms['term'] = df_terms ['term'].str.lower()
df_terms[(df_terms['party'].str.contains('N'))]

,term,party
54,ballot,N
55,mailin,N
56,mail-in,N
57,mail in,N
89,presidential debate,N
90,#debates,N
91,presidentialdebate,N
92,electoral vote,N
93,debates2020,N
94,elections2020,N


In [29]:
df_trump = df[(df['text'].str.contains('trump')) | (df['text'].str.contains('maga')) | (df['text'].str.contains('fraud'))]
df_biden = df[(df['text'].str.contains('biden')) | (df['text'].str.contains('kamala'))| (df['text'].str.contains('harris'))]

In [30]:
df_topic = df[(df['text'].str.contains('abortion'))]

In [31]:
states_dict_trump = {}
for state in stateCodes:
    states_dict_trump[stateMapping[state]] = df_trump[df_trump['location'].str.contains(state+ '|' + stateMapping[state])]
    
    
states_dict_biden = {}
for state in stateCodes:
    states_dict_biden[stateMapping[state]] = df_biden[df_biden['location'].str.contains(state+ '|' + stateMapping[state])]

In [32]:
(170 * 40000) / 1000000

6.8

In [33]:
count = 0
for state in states:
    count+= len(states_dict_biden[state])
count

6390

In [34]:
count = 0
for state in states:
    count+= len(states_dict_trump[state])
count

12040

In [21]:
states_dict_trump['New York']

,id,text,location,place,country,retweeted,retweet_count,favorited,favorite_count,language,date,verified,sentiment_scores,sentiment
14,1289350111017881600,@realjohnbriscoe @realdonaldtrump yup the greatest mass murder in history,"Manhattan, NY",NaN,NaN,False,0,False,5,en,08-01-00,False,"{'neg': 0.296, 'neu': 0.44, 'pos': 0.264, 'compound': -0.128}",Negative
21,1289350111986778114,rt @mmpadellan: @realdonaldtrump you are doing putin's bidding by moving our troops out of germany and weakening nato.\n\nyou are a traitorou…,"New York, Cape Cod MA",NaN,NaN,False,2787,False,0,en,08-01-00,True,"{'neg': 0.103, 'neu': 0.897, 'pos': 0.0, 'compound': -0.3182}",Negative
116,1289350143259447297,@dpwimm @realdonaldtrump china owes the world a lot!!!!,"Brooklyn, NY",NaN,NaN,False,0,False,0,en,08-01-00,False,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",Neutral
136,1289350148829507584,"rt @jelle_simons: @mmpadellan @realdonaldtrump man who thinks windmills cause cancer, suggested bleach cures covid, confused nepal and bhut…","New York, Cape Cod MA",NaN,NaN,False,841,False,0,en,08-01-00,True,"{'neg': 0.295, 'neu': 0.705, 'pos': 0.0, 'compound': -0.7717}",Negative
141,1289350127111606278,"@realdonaldtrump don't let trump fool you, dems passed an additional $1,200 stimulus check + $600 unemployment enhancement back in may (2 months ago!) \n\nit's the republicans who have been dragging their feet and only started on their bill this week. https://t.co/mio0m0aztn","New York, NY",NaN,NaN,False,1,False,3,en,08-01-00,False,"{'neg': 0.073, 'neu': 0.872, 'pos': 0.055, 'compound': -0.1989}",Negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49690,1289365168724627456,"rt @hkrassenstein: @realdonaldtrump no, you never beat obama! you never could and never will. you can’t even hold obama’s jock strap.","New York, USA",NaN,NaN,False,1108,False,0,en,08-01-00,False,"{'neg': 0.111, 'neu': 0.889, 'pos': 0.0, 'compound': -0.3595}",Negative
49730,1289365180867125248,@gop @realdonaldtrump https://t.co/6wztb8ptuf,NYC,NaN,NaN,False,0,False,0,qme,08-01-00,False,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",Neutral
49733,1289365181596803072,rt @projectlincoln: @realdonaldtrump https://t.co/gmjfa79b6j,"New York, NY",NaN,NaN,False,4572,False,0,qme,08-01-00,False,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",Neutral
49817,1289365198021652480,"rt @realdonaldtrump: the democrats are holding back the $1,200 to $3,400 (family of four) checks that were ready to be sent out!","New York, USA",NaN,NaN,False,32721,False,0,en,08-01-00,False,"{'neg': 0.0, 'neu': 0.883, 'pos': 0.117, 'compound': 0.4199}",Positive


In [17]:
df_topic = df[(df['text'].str.contains('abortion'))]
states_dict_topic = {}
for state in stateCodes:
    states_dict_topic[stateMapping[state]] = df_topic[df_topic['location'].str.contains(state+ '|' + stateMapping[state])]
count = 0
for state in states:
    count+= len(states_dict_topic[state])
count

4

In [67]:
len(df)

49627

In [18]:
len(df[(df['text'].str.contains('abortion'))])

11